# Table joiner script

In [1]:
# load required libraries and set working directory
import sys
import os
import csv
import re

os.chdir("")
masterTable = ".txt" #this is the Compounds table from CD after opening in excel and saving as a tab-delimited .txt file
tableList = [""]

reportFile = "Name.txt" # change this to whatever you want the output to be

primaryIndexHeader = "row_id"
idRegEx = r'.*'

join_ppm_tolerance = 10
join_RT_tolerance = 10 #Maximum RT Tolerance in sec

output_string = ""

def tab2dict(tableName, primaryIndexHeader, idRegEx):
    with open(tableName,encoding = "ISO-8859-1") as f:
        reader=csv.reader(f,delimiter='\t')
        headers = next(reader)
        #print(headers)
        numheaders = len(headers)
        columnindices = list(range(0, numheaders))
        
        header_indices = dict(zip(headers, columnindices))
        
        f.seek(0)
        next(f) # skip headings
        reader=csv.reader(f,delimiter='\t')

        tabdict = {}
        for line in reader:
            
            identifiers = re.findall(idRegEx, line[header_indices[primaryIndexHeader]], re.IGNORECASE)         

            for identifier in identifiers:
                tabdict[identifier.lower()] = {}
                tabdict[identifier.lower()]["dict"] = dict(zip(headers, line))
                tabdict[identifier.lower()]["list"] = line
    
    result = {}
    result["headers"] = headers
    result["tabdict"] = tabdict
    result["header_indices"] = header_indices
    result["tableName"] = tableName
    return result
            

masterDict = tab2dict(masterTable, primaryIndexHeader, idRegEx)   


tableDicts = {}

joinedHeaders = masterDict["headers"]

for tableName in tableList:
    
    tableDicts[tableName] = tab2dict(tableName, "row_id", idRegEx)
    joinedHeaders.extend(tableDicts[tableName]["headers"])

joinedRows = []    
joinedRows.append("\t".join(joinedHeaders))

for rowID, value in masterDict["tabdict"].items():    
    masterDict["tabdict"][rowID]["matches"] = {}
    
    if rowID == "":
        continue
        
    print("Processing RowID: " + rowID)
    joinedRow = masterDict["tabdict"][rowID]["list"]

    rt1 = float(masterDict["tabdict"][rowID]["dict"]["RT [min]"]) * 60

    mw1 = float(masterDict["tabdict"][rowID]["dict"]["m/z"]) 

    for tableName in tableList:
        
        minRTDiff = join_RT_tolerance
        
        rowInit = False
        
        for table2rowID in tableDicts[tableName]["tabdict"]:
            
            #in case we don't find a match for this row, we need to initiate the top_match with blank cells 
            if rowInit == False:
                masterDict["tabdict"][rowID]["top_match"] = [""]*len(tableDicts[tableName]["tabdict"][table2rowID]["list"])
                rowInit = True
            
            rt2 = float(tableDicts[tableName]["tabdict"][table2rowID]["dict"]["RT [min]"]) * 60
            mw2 = float(tableDicts[tableName]["tabdict"][table2rowID]["dict"]["m/z"])
            
            rtDiff = abs(rt2-rt1)
            
            mwDiff = abs(mw2-mw1)
            
            mwDiffppm = 1000000 * mwDiff / mw1;
            
            #if RT difference is less than join RT tolerance and MW ppm difference is less that join ppm tolerance and RT difference is less than the current best (smallest) RT difference, update the top match for this row of the master table from this secondary joining table
            if rtDiff < join_RT_tolerance and mwDiffppm < join_ppm_tolerance and rtDiff < minRTDiff:
                
                masterDict["tabdict"][rowID]["top_match"] = tableDicts[tableName]["tabdict"][table2rowID]["list"]
                minRTDiff = rtDiff
                rowInit = True
        
        
        joinedRow.extend(masterDict["tabdict"][rowID]["top_match"])

    joinedRows.append("\t".join(joinedRow))

output_string = "\n".join(joinedRows)

text_file = open(reportFile, "w")
text_file.write(output_string)
text_file.close()
print("Done.")

OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: ''